In [1]:
import pandas as pd
from dash import html, dcc, Input, Output
from dash.exceptions import PreventUpdate
from jupyter_dash import JupyterDash
import jupyter_dash as jd
import plotly
import dash
import plotly.express as px
import dash_bootstrap_components as dbc
import warnings
warnings.filterwarnings('ignore')

for pkg in [dash, plotly, pd, dbc, jd]:
    print(f'{pkg.__name__:-<30}v{pkg.__version__}')

dash--------------------------v2.5.0
plotly------------------------v5.4.0
pandas------------------------v1.4.2
dash_bootstrap_components-----v1.1.0
jupyter_dash------------------v0.4.2


In [2]:
def treemap(traffic_df, metric='Users', path=['Medium', 'Source']):
    """Make in interactive treemap for two data dimensions/levels.

    Parameters:
    -----------

    traffic_df : A DataFrame containing two dimensions, and one or more metrics

    metric : The selected metric which you want to analyze

    path : The names of the two dimensions that make the levels of the tree map (order is important)

    Returns:
    --------

    A plotly Figure object that can be further edited
    """
    fig = px.treemap(traffic_df, path=[px.Constant(metric)] + path, template='none', values=metric, height=650)
    template = '<b>%{label}</b><br><br>Total: %{value:,d}<br>%{percentParent:.1%} of %{parent}<br>%{percentRoot:.1%} of %{root}'
    fig.data[0]['texttemplate'] = template
    fig.data[0]['hovertemplate'] = template
    return fig

In [3]:
traffic = pd.read_csv('https://drive.google.com/u/0/uc?id=1r0VJGsX9vGTCJVKtxJDY-cETNzn7APxY&export=download')
traffic.head(3)

,Medium,Source,Users,New Users,Sessions,Bounce Rate,Pages / Session,Avg. Session Duration,Spent >= 7 minutes (Goal 1 Conversion Rate),Spent >= 7 minutes (Goal 1 Completions),Spent >= 7 minutes (Goal 1 Value)
0,organic,google,"10,437","10,168","13,215",69.48%,1.93,0:02:14,9.41%,"1,243","$1,243.00"
1,(none),(direct),"3,400","3,406","4,190",60.95%,2.06,0:02:05,7.97%,334,$334.00
2,referral,github.com,547,412,806,49.63%,3.07,0:04:05,15.76%,127,$127.00


## Convert to numbers, remove punctuation, etc.

In [4]:
for col in traffic[['Users', 'New Users', 'Sessions']]:
    traffic[col] = traffic[col].str.replace(',', '').astype(int)

traffic['Total Pageviews'] = traffic['Sessions'] * traffic['Pages / Session']

traffic['Spent >= 7 minutes (Goal 1 Completions)'] = traffic['Spent >= 7 minutes (Goal 1 Completions)'].str.replace(',','').astype(int)
traffic = traffic.rename(columns={'Spent >= 7 minutes (Goal 1 Completions)': 'Goal Completions'})

hms = traffic['Avg. Session Duration'].str.split(':')
total_seconds = hms.str[1].astype(int).mul(60).add(hms.str[2].astype(int))
traffic['Total Hours'] = traffic['Sessions'] * total_seconds / 60 / 60
traffic

,Medium,Source,Users,New Users,Sessions,Bounce Rate,Pages / Session,Avg. Session Duration,Spent >= 7 minutes (Goal 1 Conversion Rate),Goal Completions,Spent >= 7 minutes (Goal 1 Value),Total Pageviews,Total Hours
0,organic,google,10437,10168,13215,69.48%,1.93,0:02:14,9.41%,1243,"$1,243.00",25504.95,491.891667
1,(none),(direct),3400,3406,4190,60.95%,2.06,0:02:05,7.97%,334,$334.00,8631.40,145.486111
2,referral,github.com,547,412,806,49.63%,3.07,0:04:05,15.76%,127,$127.00,2474.42,54.852778
3,referral,trafficcentr.xyz,367,303,400,0.00%,2.00,0:00:10,0.00%,0,$0.00,800.00,1.111111
4,referral,t.co,363,348,405,62.47%,2.06,0:01:35,5.43%,22,$22.00,834.30,10.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,referral,yandex.by,1,1,1,100.00%,1.00,0:00:00,0.00%,0,$0.00,1.00,0.000000
119,referral,yellow-search.com,1,1,1,100.00%,1.00,0:00:00,0.00%,0,$0.00,1.00,0.000000
120,referral,zorotools.atlassian.net,1,1,2,50.00%,1.50,0:00:05,0.00%,0,$0.00,3.00,0.002778
121,web,Weekly_SEO_77,1,1,2,50.00%,3.50,0:03:10,0.00%,0,$0.00,7.00,0.105556


In [5]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.Br(), html.Br(),
    dbc.Row([
        dbc.Col(lg=1),
        dbc.Col([
            dbc.Label('Path:'),
        dcc.Dropdown(id='path',
                     options=[{'label': 'Medium/Source', 'value': 'Medium Source'},
                              {'label': 'Source/Medium', 'value': 'Source Medium'}],
                     value='Medium Source'),
        ]),
        dbc.Col([
            dbc.Label('Metric:'),
            dcc.Dropdown(id='metric',
                         options=[{'label': m, 'value': m}
                                  for m in ['Users', 'New Users', 'Sessions', 
                                            'Goal Completions', 'Total Pageviews',
                                            'Total Hours']],
                         value='Users')
        ]),
        dbc.Col(), html.Br(), html.Br(),
        dcc.Graph(id='graph')
    ])
])


@app.callback(Output('graph', 'figure'), Input('path', 'value'), Input('metric', 'value'))
def make_treemap(path, metric):
    if not path or not metric:
        raise PreventUpdate
    return treemap(traffic, metric=metric, path=path.split())

app.run_server(debug=False, height=1000)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jun/2022 16:12:44] "GET /_alive_e7171df1-fb26-46e5-bba0-48bd01b11016 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/
